<a href="https://colab.research.google.com/github/ps11061601/CCProject/blob/master/CCProject_trial2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 145113 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.17-0ubuntu2~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.17-0ubuntu2~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.17-0ubuntu2~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!pip install -q keras

In [0]:
import os
import glob
import numpy as np
from tensorflow.keras import layers
from tensorflow import keras 
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [0]:
train = pd.read_csv("preprocessed_1.csv")
y= train[['price']]
cols = train[['price','Unnamed: 0']]
X = train.drop(columns=cols)
X.head()

,neighbourhood_group,latitude,longitude,room_type,minimum_nights,number_of_reviews,availability_365
0,Brooklyn,40.64749,-73.97237,Private room,1,9,365
1,Manhattan,40.75362,-73.98377,Entire home/apt,1,45,355
2,Manhattan,40.80902,-73.94190,Private room,3,0,365
3,Brooklyn,40.68514,-73.95976,Entire home/apt,1,270,194
4,Manhattan,40.79851,-73.94399,Entire home/apt,10,9,0


In [0]:
X = pd.get_dummies(X)
X.head()

,latitude,longitude,minimum_nights,number_of_reviews,availability_365,neighbourhood_group_Bronx,neighbourhood_group_Brooklyn,neighbourhood_group_Manhattan,neighbourhood_group_Queens,neighbourhood_group_Staten Island,room_type_Entire home/apt,room_type_Private room,room_type_Shared room
0,40.64749,-73.97237,1,9,365,0,1,0,0,0,0,1,0
1,40.75362,-73.98377,1,45,355,0,0,1,0,0,1,0,0
2,40.80902,-73.94190,3,0,365,0,0,1,0,0,0,1,0
3,40.68514,-73.95976,1,270,194,0,1,0,0,0,1,0,0
4,40.79851,-73.94399,10,9,0,0,0,1,0,0,1,0,0


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [0]:
scaler = StandardScaler(copy= True, with_mean= True, with_std= True)
X_train_scaled = scaler.fit_transform(X_train)
X_train_scaled = pd.DataFrame(X_train_scaled,columns = X_train.columns)
X_train_scaled.head()

,latitude,longitude,minimum_nights,number_of_reviews,availability_365,neighbourhood_group_Bronx,neighbourhood_group_Brooklyn,neighbourhood_group_Manhattan,neighbourhood_group_Queens,neighbourhood_group_Staten Island,room_type_Entire home/apt,room_type_Private room,room_type_Shared room
0,-0.241172,-0.064946,-0.193025,-0.277198,-0.852862,-0.15029,1.195897,-0.894201,-0.359737,-0.087023,0.958818,-0.914614,-0.155111
1,2.204077,0.257030,-0.239596,-0.479451,-0.860447,-0.15029,-0.836193,1.118317,-0.359737,-0.087023,-1.042950,1.093358,-0.155111
2,-0.714824,-0.155542,-0.239596,-0.524396,-0.860447,-0.15029,1.195897,-0.894201,-0.359737,-0.087023,-1.042950,1.093358,-0.155111
3,-0.818573,0.459080,-0.193025,1.430714,1.164663,-0.15029,1.195897,-0.894201,-0.359737,-0.087023,0.958818,-0.914614,-0.155111
4,0.332197,1.231213,-0.099884,-0.232253,-0.860447,-0.15029,-0.836193,-0.894201,2.779812,-0.087023,-1.042950,1.093358,-0.155111


In [0]:
X_test_scaled = scaler.fit_transform(X_test)
X_test_scaled = pd.DataFrame(X_test_scaled,columns = X_test.columns)
X_test_scaled.head()

,latitude,longitude,minimum_nights,number_of_reviews,availability_365,neighbourhood_group_Bronx,neighbourhood_group_Brooklyn,neighbourhood_group_Manhattan,neighbourhood_group_Queens,neighbourhood_group_Staten Island,room_type_Entire home/apt,room_type_Private room,room_type_Shared room
0,-1.570803,-0.560746,-0.221959,0.870704,0.603535,-0.154158,1.199946,-0.882410,-0.371153,-0.090246,0.971865,-0.924985,-0.158957
1,-0.411113,0.939030,0.896940,-0.514644,-0.284074,-0.154158,-0.833371,-0.882410,2.694307,-0.090246,-1.028949,1.081098,-0.158957
2,0.590204,-0.826283,-0.284120,-0.134790,-0.842656,-0.154158,-0.833371,1.133261,-0.371153,-0.090246,-1.028949,1.081098,-0.158957
3,-0.393291,-1.262199,-0.284120,-0.402922,1.238634,-0.154158,-0.833371,1.133261,-0.371153,-0.090246,0.971865,-0.924985,-0.158957
4,1.240969,0.401528,-0.284120,0.155686,-0.613102,-0.154158,-0.833371,1.133261,-0.371153,-0.090246,0.971865,-0.924985,-0.158957


In [0]:
model = Sequential()
model.add(Dense(30, input_dim=13, kernel_initializer='normal', activation='relu'))
model.add(Dense(20, kernel_initializer='normal', activation='relu'))
model.add(Dense(10, kernel_initializer='normal', activation='relu'))
model.add(Dense(5, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.compile(loss='mean_squared_error', optimizer='adam')

In [0]:
model.fit(X_train_scaled, y_train,epochs=100,batch_size =50,verbose=0)

In [0]:
pred1 = model.predict(X_train_scaled)
print('RMSE TRAIN : ',np.sqrt(mean_squared_error(pred1,y_train)))

pred2 = model.predict(X_test_scaled)
print('RMSE TEST : ',np.sqrt(mean_squared_error(pred2,y_test)))


RMSE TRAIN :  229.20529559751063
RMSE TEST :  194.12012800738447
